<a href="https://colab.research.google.com/github/ArnyWu/-Preview-AI-/blob/main/week13%E4%BD%9C%E6%A5%AD_%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import numpy as np
import random

# 股票編號
stock_codes = ['6414.TW', '2385.TW', '3231.TW', '2449.TW', '3324.TWO']

# 獲取每支股票的歷史數據（過去五年的每日收盤價）
historical_data = {}
for code in stock_codes:
    stock = yf.Ticker(code)
    historical_data[code] = stock.history(period="5y")['Close']

# 計算回報率與風險，並顯示計算過程
returns = []
risks = []
print("=== 計算每支股票的回報率與風險 ===")
for code, data in historical_data.items():
    daily_returns = data.pct_change().dropna()  # 每日收益率
    annual_return = (1 + daily_returns.mean())**252 - 1  # 年化回報率
    annual_volatility = daily_returns.std() * (252**0.5)  # 年化波動率

    # 紀錄數值
    returns.append(annual_return)
    risks.append(annual_volatility)

    # 顯示計算結果
    print(f"{code}:")
    print(f"  平均每日回報率：{daily_returns.mean():.4f}")
    print(f"  年化回報率：{annual_return:.4f}")
    print(f"  年化波動率（風險）：{annual_volatility:.4f}\n")

# 基因演算法參數設定
population_size = 10   # 初始族群大小
generations = 50       # 迭代代數
mutation_rate = 0.1    # 突變率

# 初始化族群
population = np.random.randint(2, size=(population_size, len(stock_codes)))

# 適應函數計算
# 修正適應函數，確保適應值非負
def fitness(chromosome):
    selected_returns = [returns[i] for i in range(len(chromosome)) if chromosome[i] == 1]
    selected_risks = [risks[i] for i in range(len(chromosome)) if chromosome[i] == 1]
    if not selected_returns:  # 避免全部為 0 的情況
        return 0
    portfolio_return = sum(selected_returns)
    portfolio_risk = np.sqrt(sum([r**2 for r in selected_risks]))
    # 添加正偏移量，避免適應值為負
    return max(portfolio_return / portfolio_risk, 0)



# 選擇（輪盤法）
def select(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [f / total_fitness for f in fitness_scores]
    return population[np.random.choice(len(population), p=probabilities)]

# 交配
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# 突變
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# 主迭代流程
for generation in range(generations):
    fitness_scores = [fitness(chromosome) for chromosome in population]

    # 檢查適應分數是否有非正數值
    if any(f < 0 for f in fitness_scores):
        print("警告：適應值包含非正數，檢查數據或適應函數！")
        print("適應值分數：", fitness_scores)
        break

    best_fitness = max(fitness_scores)
    best_chromosome = population[fitness_scores.index(best_fitness)]
    best_return = sum([returns[i] for i in range(len(best_chromosome)) if best_chromosome[i] == 1])
    best_risk = np.sqrt(sum([risks[i]**2 for i in range(len(best_chromosome)) if best_chromosome[i] == 1]))
    best_stocks = [stock_codes[i] for i in range(len(best_chromosome)) if best_chromosome[i] == 1]

    # 輸出每代結果
    print(f"第 {generation + 1} 代:")
    print("  最佳投資組合：", best_stocks)
    print(f"  回報率：{best_return:.4f}, 風險：{best_risk:.4f}, 適應度：{best_fitness:.4f}\n")

    # 建立新族群
    new_population = []
    for _ in range(population_size // 2):
        # 選擇父母
        parent1 = select(population, fitness_scores)
        parent2 = select(population, fitness_scores)
        # 交配
        child1, child2 = crossover(parent1, parent2)
        # 突變
        child1 = mutate(child1, mutation_rate)
        child2 = mutate(child2, mutation_rate)
        new_population.extend([child1, child2])
    population = np.array(new_population)

# 最終結果
final_best_fitness = max([fitness(chromosome) for chromosome in population])
final_best_chromosome = population[[fitness(chromosome) for chromosome in population].index(final_best_fitness)]
final_best_return = sum([returns[i] for i in range(len(final_best_chromosome)) if final_best_chromosome[i] == 1])
final_best_risk = np.sqrt(sum([risks[i]**2 for i in range(len(final_best_chromosome)) if final_best_chromosome[i] == 1]))
final_best_stocks = [stock_codes[i] for i in range(len(final_best_chromosome)) if final_best_chromosome[i] == 1]

print("\n=== 最終結果 ===")
print("最佳投資組合：", final_best_stocks)
print(f"回報率：{final_best_return:.4f}, 風險：{final_best_risk:.4f}, 適應度：{final_best_fitness:.4f}")

=== 計算每支股票的回報率與風險 ===
6414.TW:
  平均每日回報率：0.0007
  年化回報率：0.1890
  年化波動率（風險）：0.3736

2385.TW:
  平均每日回報率：0.0008
  年化回報率：0.2262
  年化波動率（風險）：0.2549

3231.TW:
  平均每日回報率：0.0016
  年化回報率：0.5134
  年化波動率（風險）：0.3711

2449.TW:
  平均每日回報率：0.0015
  年化回報率：0.4455
  年化波動率（風險）：0.3583

3324.TWO:
  平均每日回報率：0.0016
  年化回報率：0.5029
  年化波動率（風險）：0.5325

第 1 代:
  最佳投資組合： ['6414.TW', '2385.TW', '3231.TW', '2449.TW', '3324.TWO']
  回報率：1.8770, 風險：0.8684, 適應度：2.1613

第 2 代:
  最佳投資組合： ['6414.TW', '2385.TW', '3231.TW', '2449.TW', '3324.TWO']
  回報率：1.8770, 風險：0.8684, 適應度：2.1613

第 3 代:
  最佳投資組合： ['2385.TW', '3231.TW', '2449.TW', '3324.TWO']
  回報率：1.6880, 風險：0.7840, 適應度：2.1531

第 4 代:
  最佳投資組合： ['6414.TW', '2385.TW', '3231.TW', '2449.TW', '3324.TWO']
  回報率：1.8770, 風險：0.8684, 適應度：2.1613

第 5 代:
  最佳投資組合： ['6414.TW', '2385.TW', '3231.TW', '2449.TW', '3324.TWO']
  回報率：1.8770, 風險：0.8684, 適應度：2.1613

第 6 代:
  最佳投資組合： ['2385.TW', '3231.TW', '2449.TW', '3324.TWO']
  回報率：1.6880, 風險：0.7840, 適應度：2.1531

第 7 代:
  最佳投資組合： ['2385.TW',